In [2]:
# %load PreProcess.py
import glob

import scipy.signal as sig
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [3]:
def feature_normalization(X):
    #X = X.astype(float)
    scaler = StandardScaler().fit(X)
    data = scaler.transform(X)
    return data

In [4]:
def DC_cancellation(Signal):
    X = sig.detrend(Signal)
    DC_Signal = X - np.mean(X)
    DC_Signal = DC_Signal / max( abs(DC_Signal))
    return DC_Signal

In [5]:
def Filter(Signal,fs):
    L_cutoff = 5
    H_cutoff = 15
    N = 3
    Wn = [L_cutoff, H_cutoff] *2 /fs
    [b, a] = sig.butter(N, Wn, 'bandpass')
    F_signal = sig.filtfilt( b, a, Signal)
    F_signal = F_signal / max( abs(F_signal))
    return F_signal

In [6]:
def Data_pro1(data, Sepstr, Norsrt):
    w = len(data.II)
    data = data[data.II != 0]
    s = len(data.II)
    print(str(w) + '/' + str(s)) if w != s else []
    if Sepstr == -1 and Norsrt == -1:
        df = data.iloc[:,np.r_[3,5]]
        df = df[df.II != 0]
        df = df[df.Label != 0] #刪除不為敗血症

        label = df.Label
        df = df.iloc[:,np.r_[0]]
    else:

        df = data.iloc[:, np.r_[3]]
        label = data.iloc[:, np.r_[5]]

    df = df.dropna()
    normdf = feature_normalization(df)
        
    categorical_labels = to_categorical(label, num_classes=2)

    del label, data, df, w,s
    return normdf, categorical_labels

In [7]:
def Data_pro2(data, onehot_encoded, N_TIME_STEPS, step, N_FEATURES,classes):
    segments = []
    labels = []

    for i in range(0, len(data) - N_TIME_STEPS, step):
        II = data[ i : i + N_TIME_STEPS, 0]
        li = onehot_encoded[ i : i + N_TIME_STEPS]
        segments.append([II])
        labels.append(li)
    del II,li,i
    return segments, labels

In [8]:
def Save_Array(group ,path , size = 2048):
    dirs = glob.glob(path + "/*.csv")
    N_FEATURES = 1
    classes = 2
    onehot_encoded =[]
    norm_df = []
    for i in range(292, len(dirs)):
        dir = dirs[i]
    #for dir in dirs:
        print('\n'+str(dirs.index(dir)) + " // " +dir)
        datas = pd.read_csv( dir, header= 0, sep = ',' )
        norm_df, onehot_encoded = Data_pro1(datas , dir.find('NonSepsis'),dir.find('Normal'))
        del datas
        
        segments, labels = Data_pro2(norm_df , onehot_encoded, size, size, N_FEATURES,classes)
        reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, 1, size, N_FEATURES)
        print('data shape')
        print(reshaped_segments.shape)
        reshaped_labels = np.asarray(labels, dtype= np.float32).reshape(-1, 1, size, classes)
        print('label shape')
        print(reshaped_labels.shape)
        print(group +'/data.npy')
        print(os.path.isfile(group +'/data.npy'))
        
        y1 = np.load(group +'/data.npy') if os.path.isfile(group +'/data.npy') else []
        np.save(group +'/data.npy', reshaped_segments) if y1 == [] else np.save(group +'/data.npy', np.concatenate((y1,reshaped_segments),axis=0))
        y2 = np.load(group +'/label.npy') if os.path.isfile(group +'/label.npy') else []
        np.save(group +'/label.npy', reshaped_labels) if y2 == [] else np.save(group +'/label.npy', np.concatenate((y2,reshaped_labels),axis=0))
        print('accumulation data shape')
        print(y1.shape) if dir != dirs[0]  else print(str(len(y1)))
        del reshaped_labels,labels, y2, reshaped_segments, segments, y1, norm_df, onehot_encoded
        
    
    y1 = np.load(group +'/data.npy')
    print('\n\n total data shape')
    print(str(y1.shape))
    print('finish')

In [8]:
Save_Array('Normal','M:/BC case/Label/Normal/*')

KeyboardInterrupt: 

In [10]:
Save_Array('Sepsis','M:/BC case/Label/Sepsis')


291 // M:/BC case/Label/Sepsis\4P215N1544865521.csv
3457500/3456908


KeyboardInterrupt: 

In [ ]:
Save_Array('NonSepsis','M:/BC case/Label/NonSepsis/*')